# TRAIN

In [14]:
from __future__ import annotations
import json
import math
import os
from pathlib import Path
from typing import Dict, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

# Configuración de rutas
JSON_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\json\big_size\train_big_size_A_B_E_K_WH_WB.json"
CSV_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\csv\train_big_size_A_B_E_K_WH_WB.csv"
PLOTS_DIR = Path(r'D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TRAIN_EDA\plots')
PLOTS_DIR.mkdir(parents=True, exist_ok=True)


# Funvciones


def safe_read_csv(path: Path) -> pd.DataFrame:
    
    req_cols = {"Image", "x1", "y1", "x2", "y2", "Label"}
    df = pd.read_csv(path)
    missing = req_cols - set(df.columns)
    if missing:
        raise ValueError(f"Faltan columnas obligatorias en el CSV: {missing}")
    return df


def safe_read_json(path: Path) -> dict:
   
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def build_image_meta(json_data: dict) -> pd.DataFrame:

    if not json_data:
        return pd.DataFrame()
    images = json_data.get("images", [])
    if not images:
        return pd.DataFrame()
    cols = ["file_name", "width", "height", "id"]
    imeta = (
        pd.DataFrame(images)[cols]
        .rename(columns={"file_name": "Image", "id": "image_id"})
        .drop_duplicates(subset=["Image"], keep="first")
    )
    return imeta


def build_cat_map(json_data: dict) -> Dict[int, str]:

    fallback = {
        1: "Alcelaphinae",
        2: "Buffalo",
        3: "Kob",
        4: "Warthog",
        5: "Waterbuck",
        6: "Elephant",
    }
    if not json_data:
        return fallback
    cats = json_data.get("categories", [])
    if not cats:
        return fallback
    cmap = {int(c.get("id")): str(c.get("name")) for c in cats if "id" in c and "name" in c}
    return cmap or fallback


def compute_bbox_stats(df: pd.DataFrame) -> pd.DataFrame:
    "Agrega columnas de ancho/alto/área de bbox y asegura no-negatividad."
    out = df.copy()
    out["bbox_w"] = (out["x2"] - out["x1"]).clip(lower=0)
    out["bbox_h"] = (out["y2"] - out["y1"]).clip(lower=0)
    out["bbox_area"] = out["bbox_w"] * out["bbox_h"]
    return out


def attach_image_dims(ann: pd.DataFrame, imeta: pd.DataFrame) -> pd.DataFrame:
    "Merge por nombre de archivo para obtener width/height de imagen."
    if imeta.empty:
        ann["img_w"] = np.nan
        ann["img_h"] = np.nan
        ann["img_area"] = np.nan
        return ann
    merged = ann.merge(imeta[["Image", "width", "height"]], on="Image", how="left")
    merged = merged.rename(columns={"width": "img_w", "height": "img_h"})
    merged["img_area"] = merged["img_w"] * merged["img_h"]
    return merged


def compute_rel_area(ann: pd.DataFrame) -> pd.DataFrame:
    "Calcula área relativa bbox/imágen, con protección ante divisiones por cero."
    out = ann.copy()
    denom = out["img_area"].replace({0: np.nan})
    out["bbox_area_frac"] = out["bbox_area"] / denom
    return out


def orientation_from_dims(w: float, h: float) -> str:
    if pd.isna(w) or pd.isna(h):
        return "unknown"
    if w > h:
        return "landscape"
    if w < h:
        return "portrait"
    return "square"



# Carga de datos

print("\n==> Cargando CSV…")
ann_df = safe_read_csv(CSV_PATH)

print("==> Cargando JSON…")
try:
    coco = safe_read_json(JSON_PATH)
except FileNotFoundError:
    coco = {}

imeta_df = build_image_meta(coco)
cat_map = build_cat_map(coco)

# Estadísticas básicas
n_rows = len(ann_df)
unique_imgs = ann_df["Image"].nunique()
unique_labels = ann_df["Label"].nunique()
print(f"Filas en CSV: {n_rows}")
print(f"Imágenes (únicas): {unique_imgs}")
print(f"Anotaciones: {n_rows}, Categorías: {unique_labels}")
print(f"Nombres de categorías (si disponibles): {sorted(set(cat_map.values()))}")


# Preparación de anotaciones

ann_df = compute_bbox_stats(ann_df)

# Añadir nombres de clases
ann_df["category_id"] = ann_df["Label"].astype(int)
ann_df["category_name"] = ann_df["category_id"].map(cat_map).fillna(ann_df["category_id"].astype(str))

# Adjuntar dimensiones de imagen desde JSON
ann_df = attach_image_dims(ann_df, imeta_df)

# Área relativa
ann_df = compute_rel_area(ann_df)


# Tablas auxiliares

objs_per_img = ann_df.groupby("Image").size().rename("n_objs").reset_index()
cat_counts = ann_df.groupby("category_name").size().sort_values(ascending=False)

# Si tenemos metadatos de dimensiones, calculamos AR y orientación por imagen
if not imeta_df.empty:
    imeta_df["aspect_ratio"] = imeta_df["width"] / imeta_df["height"]
    imeta_df["orientation"] = [orientation_from_dims(w, h) for w, h in zip(imeta_df["width"], imeta_df["height"])]
else:
    # Construimos imeta mínimo a partir de nombres de imagen presentes en el CSV
    imeta_df = pd.DataFrame({"Image": ann_df["Image"].unique()})
    imeta_df["width"] = np.nan
    imeta_df["height"] = np.nan
    imeta_df["aspect_ratio"] = np.nan
    imeta_df["orientation"] = "unknown"


# Gráficas


def save_hist(series: pd.Series, bins: int, title: str, fname: str, xlabel: str):
    plt.figure(figsize=(8, 5))
    s = series.dropna()
    if s.empty:
        plt.text(0.5, 0.5, "Sin datos", ha="center", va="center")
    else:
        plt.hist(s, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_bar(index: pd.Index, values: pd.Series, title: str, fname: str, xlabel: str = "Categoría"):
    plt.figure(figsize=(9, 5))
    plt.bar(index.astype(str), values.values)
    plt.xticks(rotation=45, ha="right")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Conteo")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_scatter_dims(df_dims: pd.DataFrame, fname: str, title: str):
    "Dispersión width vs height (submuestreo si hay muchas imágenes)."
    plt.figure(figsize=(6, 6))
    d = df_dims.dropna(subset=["width", "height"]).copy()
    if len(d) > 2000:
        d = d.sample(2000, random_state=42)
    if d.empty:
        plt.text(0.5, 0.5, "Sin dimensiones", ha="center", va="center")
    else:
        plt.scatter(d["width"], d["height"], alpha=0.5, s=8)
        # Línea de igualdad
        lim = max(d["width"].max(), d["height"].max())
        plt.plot([0, lim], [0, lim])
        plt.xlim(0, lim)
        plt.ylim(0, lim)
    plt.title(title)
    plt.xlabel("width (px)")
    plt.ylabel("height (px)")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_orientation_bar(df_dims: pd.DataFrame, fname: str, title: str):
    counts = df_dims["orientation"].value_counts().reindex(["landscape", "portrait", "square", "unknown"], fill_value=0)
    save_bar(counts.index, counts, title, fname, xlabel="Orientación")


# 1) Distribución del tamaño relativo de bbox
save_hist(
    ann_df["bbox_area_frac"],
    bins=40,
    title="TRAIN - Distribución área relativa bbox",
    fname="TRAIN_bbox_area_frac.png",
    xlabel="bbox_area / img_area",
)

# 2) Conteo por clase
save_bar(
    cat_counts.index,
    cat_counts,
    title="TRAIN - Conteo por categoría",
    fname="TRAIN_cat_counts.png",
    xlabel="Categoría",
)

# 3) Dimensiones (dispersión width vs height)
save_scatter_dims(
    imeta_df[["width", "height"]],
    fname="TRAIN_dims.png",
    title="TRAIN - Dimensiones de imágenes",
)

# 4) Objetos por imagen
save_hist(
    objs_per_img["n_objs"],
    bins=min(50, max(10, int(objs_per_img["n_objs"].max()))),
    title="TRAIN - Objetos por imagen",
    fname="TRAIN_objs_per_img.png",
    xlabel="#objetos",
)

# 5) Orientaciones
save_orientation_bar(
    imeta_df[["orientation"]],
    fname="TRAIN_orientations.png",
    title="TRAIN - Orientaciones",
)


# Resumen en consola

category_names = list(cat_map.values())
summary = {
    "Filas en CSV": n_rows,
    "Columnas": list(ann_df.columns),
    "Imágenes": int(unique_imgs),
    "IDs únicos": int(unique_imgs),
    "Anotaciones": n_rows,
    "Categorías": int(unique_labels),
    "Nombres categorías": category_names,
}
print("\n===== RESUMEN TRAIN =====")
for k, v in summary.items():
    if isinstance(v, list):
        print(f"- {k}: {', '.join(map(str, v))}")
    else:
        print(f"- {k}: {v}")

print(f"\nGráficas guardadas en: {PLOTS_DIR.resolve()}")



# Reporte en Markdown




def df_to_markdown(df: pd.DataFrame, index=False):
    "Convierte un DataFrame a tabla Markdown."
    df = df.copy()
    cols = list(df.columns)
    header = "|" + "|".join(map(str, cols)) + "|\n"
    sep    = "|" + "|".join(["---"] * len(cols)) + "|\n"
    rows = []
    for _, r in df.iterrows():
        rows.append("|" + "|".join(map(lambda x: str(x), r.values)) + "|")
    return header + sep + "\n".join(rows)

# Tablas para el reporte
cat_tbl = cat_counts.reset_index()
cat_tbl.columns = ["Categoria", "Conteo"]

dims_stats = pd.DataFrame()
if not imeta_df.empty:
    dims_stats = imeta_df[["width", "height", "aspect_ratio"]].describe().round(2)

objs_stats = objs_per_img["n_objs"].describe().round(2).to_frame(name="n_objs")

# Ruta de salida del markdown (misma carpeta que PLOTS_DIR)
REPORT_PATH = PLOTS_DIR.parent / "TRAIN_EDA_reporte.md"

# Contenido del markdown
md = []
md.append("# Reporte EDA - TRAIN\n")
md.append(f"- Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
md.append(f"- Filas en CSV: {n_rows}")
md.append(f"- Imágenes únicas: {unique_imgs}")
md.append(f"- Anotaciones: {n_rows}")
md.append(f"- Categorías: {unique_labels}")
md.append(f"- Nombres categorías: {', '.join(category_names)}\n")

md.append("## Conteo por categoría\n")
md.append(df_to_markdown(cat_tbl) + "\n")

md.append("## Estadísticas de objetos por imagen\n")
md.append("```\n" + objs_stats.to_string() + "\n```\n")

if not dims_stats.empty:
    md.append("## Estadísticas de dimensiones (width/height/aspect_ratio)\n")
    md.append(df_to_markdown(dims_stats.reset_index().rename(columns={"index": "estatistica"})) + "\n")

md.append("## Visualizaciones\n")
md.append(f"![TRAIN - Distribución área relativa bbox](plots/TRAIN_bbox_area_frac.png)\n")
md.append(f"![TRAIN - Conteo por categoría](plots/TRAIN_cat_counts.png)\n")
md.append(f"![TRAIN - Dimensiones de imágenes](plots/TRAIN_dims.png)\n")
md.append(f"![TRAIN - Objetos por imagen](plots/TRAIN_objs_per_img.png)\n")
md.append(f"![TRAIN - Orientaciones](plots/TRAIN_orientations.png)\n")

# Guardar markdown
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print(f"\nReporte Markdown generado en: {REPORT_PATH}")




==> Cargando CSV…
==> Cargando JSON…
Filas en CSV: 6962
Imágenes (únicas): 928
Anotaciones: 6962, Categorías: 6
Nombres de categorías (si disponibles): ['Alcelaphinae', 'Buffalo', 'Elephant', 'Kob', 'Warthog', 'Waterbuck']

===== RESUMEN TRAIN =====
- Filas en CSV: 6962
- Columnas: Image, x1, y1, x2, y2, Label, bbox_w, bbox_h, bbox_area, category_id, category_name, img_w, img_h, img_area, bbox_area_frac
- Imágenes: 928
- IDs únicos: 928
- Anotaciones: 6962
- Categorías: 6
- Nombres categorías: Alcelaphinae, Buffalo, Kob, Warthog, Waterbuck, Elephant

Gráficas guardadas en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TRAIN_EDA\plots

Reporte Markdown generado en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TRAIN_EDA\TRAIN_EDA_reporte.md


# TEST

In [15]:
from __future__ import annotations
import json
import math
import os
from pathlib import Path
from typing import Dict, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

# Configuración de rutas
JSON_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\json\big_size\test_big_size_A_B_E_K_WH_WB.json"
CSV_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\csv\test_big_size_A_B_E_K_WH_WB.csv"
PLOTS_DIR = Path(r'D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TEST_EDA\plots')
PLOTS_DIR.mkdir(parents=True, exist_ok=True)


# Funvciones


def safe_read_csv(path: Path) -> pd.DataFrame:
    
    req_cols = {"Image", "x1", "y1", "x2", "y2", "Label"}
    df = pd.read_csv(path)
    missing = req_cols - set(df.columns)
    if missing:
        raise ValueError(f"Faltan columnas obligatorias en el CSV: {missing}")
    return df


def safe_read_json(path: Path) -> dict:
   
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def build_image_meta(json_data: dict) -> pd.DataFrame:

    if not json_data:
        return pd.DataFrame()
    images = json_data.get("images", [])
    if not images:
        return pd.DataFrame()
    cols = ["file_name", "width", "height", "id"]
    imeta = (
        pd.DataFrame(images)[cols]
        .rename(columns={"file_name": "Image", "id": "image_id"})
        .drop_duplicates(subset=["Image"], keep="first")
    )
    return imeta


def build_cat_map(json_data: dict) -> Dict[int, str]:

    fallback = {
        1: "Alcelaphinae",
        2: "Buffalo",
        3: "Kob",
        4: "Warthog",
        5: "Waterbuck",
        6: "Elephant",
    }
    if not json_data:
        return fallback
    cats = json_data.get("categories", [])
    if not cats:
        return fallback
    cmap = {int(c.get("id")): str(c.get("name")) for c in cats if "id" in c and "name" in c}
    return cmap or fallback


def compute_bbox_stats(df: pd.DataFrame) -> pd.DataFrame:
    "Agrega columnas de ancho/alto/área de bbox y asegura no-negatividad."
    out = df.copy()
    out["bbox_w"] = (out["x2"] - out["x1"]).clip(lower=0)
    out["bbox_h"] = (out["y2"] - out["y1"]).clip(lower=0)
    out["bbox_area"] = out["bbox_w"] * out["bbox_h"]
    return out


def attach_image_dims(ann: pd.DataFrame, imeta: pd.DataFrame) -> pd.DataFrame:
    "Merge por nombre de archivo para obtener width/height de imagen."
    if imeta.empty:
        ann["img_w"] = np.nan
        ann["img_h"] = np.nan
        ann["img_area"] = np.nan
        return ann
    merged = ann.merge(imeta[["Image", "width", "height"]], on="Image", how="left")
    merged = merged.rename(columns={"width": "img_w", "height": "img_h"})
    merged["img_area"] = merged["img_w"] * merged["img_h"]
    return merged


def compute_rel_area(ann: pd.DataFrame) -> pd.DataFrame:
    "Calcula área relativa bbox/imágen, con protección ante divisiones por cero."
    out = ann.copy()
    denom = out["img_area"].replace({0: np.nan})
    out["bbox_area_frac"] = out["bbox_area"] / denom
    return out


def orientation_from_dims(w: float, h: float) -> str:
    if pd.isna(w) or pd.isna(h):
        return "unknown"
    if w > h:
        return "landscape"
    if w < h:
        return "portrait"
    return "square"



# Carga de datos

print("\n==> Cargando CSV…")
ann_df = safe_read_csv(CSV_PATH)

print("==> Cargando JSON…")
try:
    coco = safe_read_json(JSON_PATH)
except FileNotFoundError:
    coco = {}

imeta_df = build_image_meta(coco)
cat_map = build_cat_map(coco)

# Estadísticas básicas
n_rows = len(ann_df)
unique_imgs = ann_df["Image"].nunique()
unique_labels = ann_df["Label"].nunique()
print(f"Filas en CSV: {n_rows}")
print(f"Imágenes (únicas): {unique_imgs}")
print(f"Anotaciones: {n_rows}, Categorías: {unique_labels}")
print(f"Nombres de categorías (si disponibles): {sorted(set(cat_map.values()))}")


# Preparación de anotaciones

ann_df = compute_bbox_stats(ann_df)

# Añadir nombres de clases
ann_df["category_id"] = ann_df["Label"].astype(int)
ann_df["category_name"] = ann_df["category_id"].map(cat_map).fillna(ann_df["category_id"].astype(str))

# Adjuntar dimensiones de imagen desde JSON
ann_df = attach_image_dims(ann_df, imeta_df)

# Área relativa
ann_df = compute_rel_area(ann_df)


# Tablas auxiliares

objs_per_img = ann_df.groupby("Image").size().rename("n_objs").reset_index()
cat_counts = ann_df.groupby("category_name").size().sort_values(ascending=False)

# Si tenemos metadatos de dimensiones, calculamos AR y orientación por imagen
if not imeta_df.empty:
    imeta_df["aspect_ratio"] = imeta_df["width"] / imeta_df["height"]
    imeta_df["orientation"] = [orientation_from_dims(w, h) for w, h in zip(imeta_df["width"], imeta_df["height"])]
else:
    # Construimos imeta mínimo a partir de nombres de imagen presentes en el CSV
    imeta_df = pd.DataFrame({"Image": ann_df["Image"].unique()})
    imeta_df["width"] = np.nan
    imeta_df["height"] = np.nan
    imeta_df["aspect_ratio"] = np.nan
    imeta_df["orientation"] = "unknown"


# Gráficas


def save_hist(series: pd.Series, bins: int, title: str, fname: str, xlabel: str):
    plt.figure(figsize=(8, 5))
    s = series.dropna()
    if s.empty:
        plt.text(0.5, 0.5, "Sin datos", ha="center", va="center")
    else:
        plt.hist(s, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_bar(index: pd.Index, values: pd.Series, title: str, fname: str, xlabel: str = "Categoría"):
    plt.figure(figsize=(9, 5))
    plt.bar(index.astype(str), values.values)
    plt.xticks(rotation=45, ha="right")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Conteo")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_scatter_dims(df_dims: pd.DataFrame, fname: str, title: str):
    "Dispersión width vs height (submuestreo si hay muchas imágenes)."
    plt.figure(figsize=(6, 6))
    d = df_dims.dropna(subset=["width", "height"]).copy()
    if len(d) > 2000:
        d = d.sample(2000, random_state=42)
    if d.empty:
        plt.text(0.5, 0.5, "Sin dimensiones", ha="center", va="center")
    else:
        plt.scatter(d["width"], d["height"], alpha=0.5, s=8)
        # Línea de igualdad
        lim = max(d["width"].max(), d["height"].max())
        plt.plot([0, lim], [0, lim])
        plt.xlim(0, lim)
        plt.ylim(0, lim)
    plt.title(title)
    plt.xlabel("width (px)")
    plt.ylabel("height (px)")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_orientation_bar(df_dims: pd.DataFrame, fname: str, title: str):
    counts = df_dims["orientation"].value_counts().reindex(["landscape", "portrait", "square", "unknown"], fill_value=0)
    save_bar(counts.index, counts, title, fname, xlabel="Orientación")


# 1) Distribución del tamaño relativo de bbox
save_hist(
    ann_df["bbox_area_frac"],
    bins=40,
    title="TEST - Distribución área relativa bbox",
    fname="TEST_bbox_area_frac.png",
    xlabel="bbox_area / img_area",
)

# 2) Conteo por clase
save_bar(
    cat_counts.index,
    cat_counts,
    title="TEST - Conteo por categoría",
    fname="TEST_cat_counts.png",
    xlabel="Categoría",
)

# 3) Dimensiones (dispersión width vs height)
save_scatter_dims(
    imeta_df[["width", "height"]],
    fname="TEST_dims.png",
    title="TEST - Dimensiones de imágenes",
)

# 4) Objetos por imagen
save_hist(
    objs_per_img["n_objs"],
    bins=min(50, max(10, int(objs_per_img["n_objs"].max()))),
    title="TEST - Objetos por imagen",
    fname="TEST_objs_per_img.png",
    xlabel="#objetos",
)

# 5) Orientaciones
save_orientation_bar(
    imeta_df[["orientation"]],
    fname="TEST_orientations.png",
    title="TEST - Orientaciones",
)


# Resumen en consola

category_names = list(cat_map.values())
summary = {
    "Filas en CSV": n_rows,
    "Columnas": list(ann_df.columns),
    "Imágenes": int(unique_imgs),
    "IDs únicos": int(unique_imgs),
    "Anotaciones": n_rows,
    "Categorías": int(unique_labels),
    "Nombres categorías": category_names,
}
print("\n===== RESUMEN TEST =====")
for k, v in summary.items():
    if isinstance(v, list):
        print(f"- {k}: {', '.join(map(str, v))}")
    else:
        print(f"- {k}: {v}")

print(f"\nGráficas guardadas en: {PLOTS_DIR.resolve()}")



# Reporte en Markdown




def df_to_markdown(df: pd.DataFrame, index=False):
    "Convierte un DataFrame a tabla Markdown."
    df = df.copy()
    cols = list(df.columns)
    header = "|" + "|".join(map(str, cols)) + "|\n"
    sep    = "|" + "|".join(["---"] * len(cols)) + "|\n"
    rows = []
    for _, r in df.iterrows():
        rows.append("|" + "|".join(map(lambda x: str(x), r.values)) + "|")
    return header + sep + "\n".join(rows)

# Tablas para el reporte
cat_tbl = cat_counts.reset_index()
cat_tbl.columns = ["Categoria", "Conteo"]

dims_stats = pd.DataFrame()
if not imeta_df.empty:
    dims_stats = imeta_df[["width", "height", "aspect_ratio"]].describe().round(2)

objs_stats = objs_per_img["n_objs"].describe().round(2).to_frame(name="n_objs")

# Ruta de salida del markdown (misma carpeta que PLOTS_DIR)
REPORT_PATH = PLOTS_DIR.parent / "TEST_EDA_reporte.md"

# Contenido del markdown
md = []
md.append("# Reporte EDA - TEST\n")
md.append(f"- Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
md.append(f"- Filas en CSV: {n_rows}")
md.append(f"- Imágenes únicas: {unique_imgs}")
md.append(f"- Anotaciones: {n_rows}")
md.append(f"- Categorías: {unique_labels}")
md.append(f"- Nombres categorías: {', '.join(category_names)}\n")

md.append("## Conteo por categoría\n")
md.append(df_to_markdown(cat_tbl) + "\n")

md.append("## Estadísticas de objetos por imagen\n")
md.append("```\n" + objs_stats.to_string() + "\n```\n")

if not dims_stats.empty:
    md.append("## Estadísticas de dimensiones (width/height/aspect_ratio)\n")
    md.append(df_to_markdown(dims_stats.reset_index().rename(columns={"index": "estatistica"})) + "\n")

md.append("## Visualizaciones\n")
md.append(f"![TEST - Distribución área relativa bbox](plots/TEST_bbox_area_frac.png)\n")
md.append(f"![TEST - Conteo por categoría](plots/TEST_cat_counts.png)\n")
md.append(f"![TEST - Dimensiones de imágenes](plots/TEST_dims.png)\n")
md.append(f"![TEST - Objetos por imagen](plots/TEST_objs_per_img.png)\n")
md.append(f"![TEST - Orientaciones](plots/TEST_orientations.png)\n")

# Guardar markdown
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print(f"\nReporte Markdown generado en: {REPORT_PATH}")


==> Cargando CSV…
==> Cargando JSON…
Filas en CSV: 2299
Imágenes (únicas): 258
Anotaciones: 2299, Categorías: 6
Nombres de categorías (si disponibles): ['Alcelaphinae', 'Buffalo', 'Elephant', 'Kob', 'Warthog', 'Waterbuck']

===== RESUMEN TEST =====
- Filas en CSV: 2299
- Columnas: Image, x1, y1, x2, y2, Label, bbox_w, bbox_h, bbox_area, category_id, category_name, img_w, img_h, img_area, bbox_area_frac
- Imágenes: 258
- IDs únicos: 258
- Anotaciones: 2299
- Categorías: 6
- Nombres categorías: Alcelaphinae, Buffalo, Kob, Warthog, Waterbuck, Elephant

Gráficas guardadas en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TEST_EDA\plots

Reporte Markdown generado en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\TEST_EDA\TEST_EDA_reporte.md


# VAL

In [16]:
from __future__ import annotations
import json
import math
import os
from pathlib import Path
from typing import Dict, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

# Configuración de rutas
JSON_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\json\big_size\val_big_size_A_B_E_K_WH_WB.json"
CSV_PATH = r"D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\general_dataset\groundtruth\csv\val_big_size_A_B_E_K_WH_WB.csv"
PLOTS_DIR = Path(r'D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\VAL_EDA\plots')
PLOTS_DIR.mkdir(parents=True, exist_ok=True)


# Funvciones


def safe_read_csv(path: Path) -> pd.DataFrame:
    
    req_cols = {"Image", "x1", "y1", "x2", "y2", "Label"}
    df = pd.read_csv(path)
    missing = req_cols - set(df.columns)
    if missing:
        raise ValueError(f"Faltan columnas obligatorias en el CSV: {missing}")
    return df


def safe_read_json(path: Path) -> dict:
   
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def build_image_meta(json_data: dict) -> pd.DataFrame:

    if not json_data:
        return pd.DataFrame()
    images = json_data.get("images", [])
    if not images:
        return pd.DataFrame()
    cols = ["file_name", "width", "height", "id"]
    imeta = (
        pd.DataFrame(images)[cols]
        .rename(columns={"file_name": "Image", "id": "image_id"})
        .drop_duplicates(subset=["Image"], keep="first")
    )
    return imeta


def build_cat_map(json_data: dict) -> Dict[int, str]:

    fallback = {
        1: "Alcelaphinae",
        2: "Buffalo",
        3: "Kob",
        4: "Warthog",
        5: "Waterbuck",
        6: "Elephant",
    }
    if not json_data:
        return fallback
    cats = json_data.get("categories", [])
    if not cats:
        return fallback
    cmap = {int(c.get("id")): str(c.get("name")) for c in cats if "id" in c and "name" in c}
    return cmap or fallback


def compute_bbox_stats(df: pd.DataFrame) -> pd.DataFrame:
    "Agrega columnas de ancho/alto/área de bbox y asegura no-negatividad."
    out = df.copy()
    out["bbox_w"] = (out["x2"] - out["x1"]).clip(lower=0)
    out["bbox_h"] = (out["y2"] - out["y1"]).clip(lower=0)
    out["bbox_area"] = out["bbox_w"] * out["bbox_h"]
    return out


def attach_image_dims(ann: pd.DataFrame, imeta: pd.DataFrame) -> pd.DataFrame:
    "Merge por nombre de archivo para obtener width/height de imagen."
    if imeta.empty:
        ann["img_w"] = np.nan
        ann["img_h"] = np.nan
        ann["img_area"] = np.nan
        return ann
    merged = ann.merge(imeta[["Image", "width", "height"]], on="Image", how="left")
    merged = merged.rename(columns={"width": "img_w", "height": "img_h"})
    merged["img_area"] = merged["img_w"] * merged["img_h"]
    return merged


def compute_rel_area(ann: pd.DataFrame) -> pd.DataFrame:
    "Calcula área relativa bbox/imágen, con protección ante divisiones por cero."
    out = ann.copy()
    denom = out["img_area"].replace({0: np.nan})
    out["bbox_area_frac"] = out["bbox_area"] / denom
    return out


def orientation_from_dims(w: float, h: float) -> str:
    if pd.isna(w) or pd.isna(h):
        return "unknown"
    if w > h:
        return "landscape"
    if w < h:
        return "portrait"
    return "square"



# Carga de datos

print("\n==> Cargando CSV…")
ann_df = safe_read_csv(CSV_PATH)

print("==> Cargando JSON…")
try:
    coco = safe_read_json(JSON_PATH)
except FileNotFoundError:
    coco = {}

imeta_df = build_image_meta(coco)
cat_map = build_cat_map(coco)

# Estadísticas básicas
n_rows = len(ann_df)
unique_imgs = ann_df["Image"].nunique()
unique_labels = ann_df["Label"].nunique()
print(f"Filas en CSV: {n_rows}")
print(f"Imágenes (únicas): {unique_imgs}")
print(f"Anotaciones: {n_rows}, Categorías: {unique_labels}")
print(f"Nombres de categorías (si disponibles): {sorted(set(cat_map.values()))}")


# Preparación de anotaciones

ann_df = compute_bbox_stats(ann_df)

# Añadir nombres de clases
ann_df["category_id"] = ann_df["Label"].astype(int)
ann_df["category_name"] = ann_df["category_id"].map(cat_map).fillna(ann_df["category_id"].astype(str))

# Adjuntar dimensiones de imagen desde JSON
ann_df = attach_image_dims(ann_df, imeta_df)

# Área relativa
ann_df = compute_rel_area(ann_df)


# Tablas auxiliares

objs_per_img = ann_df.groupby("Image").size().rename("n_objs").reset_index()
cat_counts = ann_df.groupby("category_name").size().sort_values(ascending=False)

# Si tenemos metadatos de dimensiones, calculamos AR y orientación por imagen
if not imeta_df.empty:
    imeta_df["aspect_ratio"] = imeta_df["width"] / imeta_df["height"]
    imeta_df["orientation"] = [orientation_from_dims(w, h) for w, h in zip(imeta_df["width"], imeta_df["height"])]
else:
    # Construimos imeta mínimo a partir de nombres de imagen presentes en el CSV
    imeta_df = pd.DataFrame({"Image": ann_df["Image"].unique()})
    imeta_df["width"] = np.nan
    imeta_df["height"] = np.nan
    imeta_df["aspect_ratio"] = np.nan
    imeta_df["orientation"] = "unknown"


# Gráficas


def save_hist(series: pd.Series, bins: int, title: str, fname: str, xlabel: str):
    plt.figure(figsize=(8, 5))
    s = series.dropna()
    if s.empty:
        plt.text(0.5, 0.5, "Sin datos", ha="center", va="center")
    else:
        plt.hist(s, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_bar(index: pd.Index, values: pd.Series, title: str, fname: str, xlabel: str = "Categoría"):
    plt.figure(figsize=(9, 5))
    plt.bar(index.astype(str), values.values)
    plt.xticks(rotation=45, ha="right")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Conteo")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_scatter_dims(df_dims: pd.DataFrame, fname: str, title: str):
    "Dispersión width vs height (submuestreo si hay muchas imágenes)."
    plt.figure(figsize=(6, 6))
    d = df_dims.dropna(subset=["width", "height"]).copy()
    if len(d) > 2000:
        d = d.sample(2000, random_state=42)
    if d.empty:
        plt.text(0.5, 0.5, "Sin dimensiones", ha="center", va="center")
    else:
        plt.scatter(d["width"], d["height"], alpha=0.5, s=8)
        # Línea de igualdad
        lim = max(d["width"].max(), d["height"].max())
        plt.plot([0, lim], [0, lim])
        plt.xlim(0, lim)
        plt.ylim(0, lim)
    plt.title(title)
    plt.xlabel("width (px)")
    plt.ylabel("height (px)")
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / fname, dpi=180)
    plt.close()


def save_orientation_bar(df_dims: pd.DataFrame, fname: str, title: str):
    counts = df_dims["orientation"].value_counts().reindex(["landscape", "portrait", "square", "unknown"], fill_value=0)
    save_bar(counts.index, counts, title, fname, xlabel="Orientación")


# 1) Distribución del tamaño relativo de bbox
save_hist(
    ann_df["bbox_area_frac"],
    bins=40,
    title="VAL - Distribución área relativa bbox",
    fname="VAL_bbox_area_frac.png",
    xlabel="bbox_area / img_area",
)

# 2) Conteo por clase
save_bar(
    cat_counts.index,
    cat_counts,
    title="VAL - Conteo por categoría",
    fname="VAL_cat_counts.png",
    xlabel="Categoría",
)

# 3) Dimensiones (dispersión width vs height)
save_scatter_dims(
    imeta_df[["width", "height"]],
    fname="VAL_dims.png",
    title="VAL - Dimensiones de imágenes",
)

# 4) Objetos por imagen
save_hist(
    objs_per_img["n_objs"],
    bins=min(50, max(10, int(objs_per_img["n_objs"].max()))),
    title="VAL - Objetos por imagen",
    fname="VAL_objs_per_img.png",
    xlabel="#objetos",
)

# 5) Orientaciones
save_orientation_bar(
    imeta_df[["orientation"]],
    fname="VAL_orientations.png",
    title="VAL - Orientaciones",
)


# Resumen en consola

category_names = list(cat_map.values())
summary = {
    "Filas en CSV": n_rows,
    "Columnas": list(ann_df.columns),
    "Imágenes": int(unique_imgs),
    "IDs únicos": int(unique_imgs),
    "Anotaciones": n_rows,
    "Categorías": int(unique_labels),
    "Nombres categorías": category_names,
}
print("\n===== RESUMEN VAL =====")
for k, v in summary.items():
    if isinstance(v, list):
        print(f"- {k}: {', '.join(map(str, v))}")
    else:
        print(f"- {k}: {v}")

print(f"\nGráficas guardadas en: {PLOTS_DIR.resolve()}")



# Reporte en Markdown




def df_to_markdown(df: pd.DataFrame, index=False):
    "Convierte un DataFrame a tabla Markdown."
    df = df.copy()
    cols = list(df.columns)
    header = "|" + "|".join(map(str, cols)) + "|\n"
    sep    = "|" + "|".join(["---"] * len(cols)) + "|\n"
    rows = []
    for _, r in df.iterrows():
        rows.append("|" + "|".join(map(lambda x: str(x), r.values)) + "|")
    return header + sep + "\n".join(rows)

# Tablas para el reporte
cat_tbl = cat_counts.reset_index()
cat_tbl.columns = ["Categoria", "Conteo"]

dims_stats = pd.DataFrame()
if not imeta_df.empty:
    dims_stats = imeta_df[["width", "height", "aspect_ratio"]].describe().round(2)

objs_stats = objs_per_img["n_objs"].describe().round(2).to_frame(name="n_objs")

# Ruta de salida del markdown (misma carpeta que PLOTS_DIR)
REPORT_PATH = PLOTS_DIR.parent / "VAL_EDA_reporte.md"

# Contenido del markdown
md = []
md.append("# Reporte EDA - VAL\n")
md.append(f"- Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
md.append(f"- Filas en CSV: {n_rows}")
md.append(f"- Imágenes únicas: {unique_imgs}")
md.append(f"- Anotaciones: {n_rows}")
md.append(f"- Categorías: {unique_labels}")
md.append(f"- Nombres categorías: {', '.join(category_names)}\n")

md.append("## Conteo por categoría\n")
md.append(df_to_markdown(cat_tbl) + "\n")

md.append("## Estadísticas de objetos por imagen\n")
md.append("```\n" + objs_stats.to_string() + "\n```\n")

if not dims_stats.empty:
    md.append("## Estadísticas de dimensiones (width/height/aspect_ratio)\n")
    md.append(df_to_markdown(dims_stats.reset_index().rename(columns={"index": "estatistica"})) + "\n")

md.append("## Visualizaciones\n")
md.append(f"![VAL - Distribución área relativa bbox](plots/VAL_bbox_area_frac.png)\n")
md.append(f"![VAL - Conteo por categoría](plots/VAL_cat_counts.png)\n")
md.append(f"![VAL - Dimensiones de imágenes](plots/VAL_dims.png)\n")
md.append(f"![VAL - Objetos por imagen](plots/VAL_objs_per_img.png)\n")
md.append(f"![VAL - Orientaciones](plots/VAL_orientations.png)\n")

# Guardar markdown
with open(REPORT_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print(f"\nReporte Markdown generado en: {REPORT_PATH}")


==> Cargando CSV…
==> Cargando JSON…
Filas en CSV: 978
Imágenes (únicas): 111
Anotaciones: 978, Categorías: 6
Nombres de categorías (si disponibles): ['Alcelaphinae', 'Buffalo', 'Elephant', 'Kob', 'Warthog', 'Waterbuck']

===== RESUMEN VAL =====
- Filas en CSV: 978
- Columnas: Image, x1, y1, x2, y2, Label, bbox_w, bbox_h, bbox_area, category_id, category_name, img_w, img_h, img_area, bbox_area_frac
- Imágenes: 111
- IDs únicos: 111
- Anotaciones: 978
- Categorías: 6
- Nombres categorías: Alcelaphinae, Buffalo, Kob, Warthog, Waterbuck, Elephant

Gráficas guardadas en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\VAL_EDA\plots

Reporte Markdown generado en: D:\Julian\Estudio\Maestria Inteligencia Artificial\Materias\PROYECTO - DESARROLLO Y DESPLIEGUE DE SOLUCIONES\PROYECTO DE GRADO\DATA SET\EDA\VAL_EDA\VAL_EDA_reporte.md
